# Modeling a System and an Intervention

The Forest Fire Cellular Automaton is a simple computational model used to study the dynamics of forest fires. 

The Forest Fire Cellular Automaton model typically has three states for each cell: empty, tree, and burning. At each time step, the model updates the state of each cell based on the following rules:

1. A burning cell turns into an empty cell.
2. A tree will become a burning cell if at least one neighbor is burning.
3. An empty space becomes a tree with probability `p`.
4. A tree ignites, turning into a burning cell with probability `f`, regardless of its neighboring cells.

In this model, `p` is the tree growth probability and `f` is the lightning strike probability, representing the chance of a tree spontaneously catching fire. These probabilities are typically quite small.

The Forest Fire model was introduced by Drossel and Schwabl in 1992. The model is interesting because it can generate clusters of trees that are power-law distributed, meaning that there are many small clusters and a few large ones, which is a feature observed in many natural and social systems. As such, the Forest Fire model has been used to study not only the dynamics of forest fires but also other phenomena that exhibit similar patterns, such as the spread of diseases or information through a network.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the states
EMPTY, TREE, BURNING = 0, 1, 2

# Define the probabilities
p, f = 0.03, 0.001  # probabilities of tree growth and fire

# Grid size
size = (100, 100)

# Initialize grid: all cells start as empty
grid = np.zeros(size, dtype=int)

def update(grid):
    new_grid = grid.copy()
    for i in range(size[0]):
        for j in range(size[1]):
            if grid[i, j] == EMPTY and np.random.rand() < p:
                new_grid[i, j] = TREE
            elif grid[i, j] == TREE:
                if np.random.rand() < f:
                    new_grid[i, j] = BURNING
                elif any(grid[ii, jj] == BURNING for ii in range(max(i-1, 0), min(i+2, size[0]))
                                               for jj in range(max(j-1, 0), min(j+2, size[1]))):
                    new_grid[i, j] = BURNING
            elif grid[i, j] == BURNING:
                new_grid[i, j] = EMPTY
    return new_grid

In [ ]:
from matplotlib.colors import ListedColormap

cmap = ListedColormap(['black', 'green', 'orange'])

In [ ]:
from IPython import display
import time

n_steps = 10

for _ in range(n_steps):
    grid = update(grid)
    display.clear_output(wait=True)
    plt.matshow(grid, cmap=cmap)
    plt.show()
    time.sleep(0.05)